# Lesson4 ニューラル翻訳モデルを作ってみよう（Seq2Seq, Attention）

## 目次

- Section1 解説
    - 1.1 言語モデル
    - 1.2 単語のベクトル化と分散表現
    - 1.3 系列変換モデル（Seq2Seqモデル）
    - 1.4 Functional API
    - Checkクイズ
- Section2 実装①
    - 2.0 データの用意
    - 2.1 モデル構築
    - 2.2 モデルの学習
    - 2.3 モデルによる予測
    - 2.4 モデルの可視化
    - 2.5 機械翻訳の評価について（補足）
- Section3 精度向上Tips
    - 3.1 Attention機構
    - Checkクイズ
- Section4 実装②
    - 4.0 データの用意
    - 4.1 モデル構築
    - 4.2 モデルの学習
    - 4.3 モデルによる予測
    - 4.4 モデルの可視化
- Section5 ケーススタディ
    - 5.1 文書要約・対話システム
    - 5.2 音声・画像への適用
    - 5.3 スケッチの自動描画
- Homework

## Section1 解説

今回は、深層学習を使用した翻訳モデルを作成します。

そこで、まず機械学習における言語の扱いとして言語モデルについて触れたのち、各単語の扱い方を解説します。

その後、翻訳モデルとして深層学習で頻繁に使用される系列変換モデルについて解説します。

### 1.1 言語モデル

言語モデルとは、ある文章が生成される過程を確率的にモデル化したものです。

この言語モデルを獲得することで、文章の尤もらしさを測ったり、また尤もらしい文章を生成したりすることが可能になります。

もう少し具体的には、文章を単語ごとに分割してならべ、その単語列が生成される確率をモデル化します。

（なお、本来要素の決め方は自由で、1文字ごとなどで分割し、文章を文字列としてみた時の生成確率をモデル化することもあります）

各単語の生成は、周囲の単語による条件付き確率分布によって決まると言えます。（ある単語の出現確率は周囲の単語に依存すると考えられるため）

例えば、 

* P(晴れ｜今日の天気は＿＿です。)：「今日の天気は＿＿です。」の空欄の単語が「晴れ」である確率
* P(晴れ｜あの＿＿の名前はジョンです。)：「あの＿＿の名前はジョンです。」の空欄の単語が「晴れ」である確率

は、同じ「晴れ」の出現確率でも全く異なる値を取ると予想でき、確かに周囲の単語に依存した条件付き確率を考える必要があると分かります。

したがって、周囲の単語による条件付き確率分布を各条件下でモデル化していくことが、文章全体の生成モデルを考えることに相当します。

こうした条件付き確率分布を求めることができれば、逐次的に単語の生成を行うことで、尤もらしい文章の生成が可能になるわけです。

このモデル化においてニューラルネットワークを用いたものを**ニューラル言語モデル**と呼び、深層学習の発展とともに近年注目されています。

ニューラル言語モデルには大きく分けて次の二つがあります。

* **順伝播型ニューラル言語モデル（FFNN言語モデル）**：t番目の単語予想に直前の数単語分（固定長）だけ用いるモデルで、Denseレイヤーを用いる
* **再帰型ニューラル言語モデル（RNN言語モデル）**：t番目の単語予想にそれまでの系列全てを用いるモデルで、RNNやLSTMなどが相当する

ここでは、可変長入力に対応しやすく、系列データにふさわしいRNN言語モデルに注目していきます。

### 1.2 単語のベクトル化と分散表現

言語処理では、単語を扱うわけですが、これはそのままでは数学的・計算的に扱いにくいため、**数値化**や**ベクトル化**をします。

単純な数値化として出現順や頻度順での番号割り振りが考えられますが、Kerasでは`keras.preprocessing.text.Tokenizer`クラスで簡単に実行可能です。

https://keras.io/ja/preprocessing/text/#tokenizer

```python
keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                    lower=True, split=" ", char_level=False)
```

引数は次の通りです。

* num_words：利用する単語の最大数（指定するとデータセット中の頻度上位num_wordsの単語のみ使用）．
* filters：句読点などフィルタする文字のリスト
* lower：テキストを小文字に強制するか
* split：単語を分割するセパレータ
* char_level：文字ごとに分割・数値化するか

主なメソッドは次の通りです。

* fit_on_texts(texts)：入力＝学習に使う文章のリスト、出力＝なし
* texts_to_sequences(texts)：入力＝数値化する文章のリスト、出力＝数値化された文章のリスト

より詳しくは、https://keras.io/ja/preprocessing/text/#tokenizer を参照してください。

----

ベクトル化には様々な手法が存在します。もっとも単純なものは、one-hot表現に変換するものです。

ですが、この方法では、単語同士の意味の近さが含まれない不便な表現となってしまいます。

**分散表現**は単語同士の意味の近さを機械学習でモデル化したうえで、単語をベクトル化するものです。

分散表現の比較的単純なものとして、**埋め込み（Embedding）**と呼ばれるものがあります。

これはone-hot表現よりも小さい次元に特徴量を圧縮してベクトル化しますが、この圧縮の仕方はニューラルネットワークで学習します。

（one_hot表現では理想的には全ての単語分の次元が必要になりますが、Embeddingを用いることでより手ごろな次元で扱うことができるわけです。）

one_hot表現のデータをEmbeddingによって変換する際には、下図のような行列ベクトル積が行われます。($n>m$)

![Embedding](figures/embedding.png)

KerasではこのEmbeddingも`keras.layers.embeddings.Embedding`として使用できます。

※one_hot表現への変換とEmbeddingによる変換は独立していることも多いですが、Kerasでは統合されており、数値化された系列をそのまま入力します。

https://keras.io/ja/layers/embeddings/

```python
keras.layers.embeddings.Embedding(input_dim, output_dim,
                                  embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None,
                                  embeddings_constraint=None, mask_zero=False, input_length=None)
```

引数は次の通りです。

* input_dim: 単語数（＝入力データの最大インデックス + 1）
* output_dim: 出力次元（何次元に圧縮するか）
* embeddings_{initializer, regularizer, constraint}: embeddings行列のInitializers, Regularizers, Constraints
* mask_zero: 入力系列中の0をパディング（系列の長さを統一するために追加される無意味な要素）と解釈し、無視するか
* input_length: 入力の系列長

他にも分散表現を実現する手法として、俗に**word2vec**と呼ばれるものがあります。

skip-gramやCBoWというモデルを総称したもので、詳しく知りたい方は調べてみるとよいでしょう。

### 1.3 系列変換モデル

言語処理のモデルとしては、IMDB（感情ラベル付き映画レビューのデータセット）を用いた文章感情の分類などといった分類タスクももちろんあります。

ですが、もっとも現実に有用なものは、ある文章を受けて、異なる文章を生成するようなモデルでしょう。

こうしたモデルを**系列変換モデル（Seq2Seqモデル）**と言います。

大まかな構成としては下図のとおりで、これは役割の違いで2つのモジュールに分けて考えることが可能です。

* **符号化器**（左3ユニット）：入力系列を受け取って抽象化します
* **復号化器**（右5ユニット）：抽象化された入力系列を加味しつつ、真の出力系列を元に各々1つ先の単語を出力します

この符号化器と復号化器という2モジュールからなっているという側面に注目して、Seq2Seqモデルをencorder-decorderモデルと呼ぶこともあります。

![Seq2Seq](figures/seq2seq.png "Seq2Seq")

出典：I. Sutskever, O. Vinyals, and Q. Le, "Sequence to Sequence Learning with Neural Networks," NIPS 2014. ( https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf )

実際にSeq2Seqモデルを構成する場合には大まかに以下のようなレイヤー構成にします。

1. **符号化器Embeddingレイヤー**：特徴量変換（入力系列のone_hot表現→埋め込み表現）
2. **符号化器再帰レイヤー**：入力系列を"抽象化"（最終的な隠れ状態ベクトルの取得が目的、符号化器の途中の出力系列には興味がない）
3. **復号化器Embeddingレイヤー**：特徴量変換（(5で生成された)直前の出力単語のone_hot表現→埋め込み表現）
4. **復号化器再帰レイヤー**：抽象化した入力系列を加味しながら（状態ベクトルの初期値として使う）、現在の単語の1つ先の単語を出力
5. **復号化器出力レイヤー**：復号化器再帰レイヤーの出力系列をもとにして目的の出力系列に変換する（隠れ状態ベクトル表現→one-hot表現）

つまり、RNN言語モデルで符号化器と復号化器の骨格を構成し、入力や出力との間をEmbeddingレイヤー（&Denseレイヤー）で取り持っているわけです。

再帰レイヤーにはRNNやLSTMのほかにもGRUなどがありますし、単方向か双方向か、何層積み重ねるかなど幅広い選択肢があり、工夫が求められる部分です。

このSeq2Seqモデルを用いれば、機械翻訳や文書要約、質問応答などの文章入力に対して文章出力が求められるタスクに生かすことが可能です。

こうしたSeq2Seqモデルの作成に当たっては、再帰レイヤーで①隠れ状態ベクトルが取得でき、②出力系列の取得ができる必要がありあます。

具体的には、
* ①隠れ状態ベクトル（LSTMの$c_t,h_t$に相当）を取得：引数に`return_state=True`を指定
* ②出力系列を取得：引数に`return_sequences=True`を指定

とすればよく、LSTMレイヤーを生成する際の引数として指定します。

実際のコード例については、Section2の実装コードを参照してください。

LSTMについてはLesson3に既出ですので、詳細はLesson3を復習してみてください。

### 1.4 Functional API

先程Seq2Seqモデルを紹介しましたが、実はこのSeq2SeqモデルはこれまでのSequentialクラスによるモデル構築では実現できません。

Sequentialクラスを用いる場合はadd関数を使用して簡単にモデルを構築可能である一方で、途中に分岐や合流があるような複雑なモデルは作成できません。

こうしたより複雑なモデルの構築方法が**Functional API**です。Seq2Seqモデルもこちらの方法によって実装可能になります。

このFunctional APIの実装上の特徴は、

* **Inputレイヤー**から構築を始める
* 各レイヤーの返り値（テンソル）を次のレイヤーの入力として順々に構築していく
* **keras.models.Modelクラス**に入力と出力を指定することでモデルを生成

といった点が挙げられます。

といっても、一度Modelクラスのインスタンスを作ってしまえば、後の学習等はSequentialクラスによる場合と同様です。

ここではまず概要を知ってもらうため、普通の全結合ネットワークによる例を下に示しておきます。

後ほどSeq2Seqモデルの実装時にも登場するので、そこで本格的な使用時の感覚も持ってもらえればと思います。

https://keras.io/ja/getting-started/functional-api-guide/

https://keras.io/ja/models/model/

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist       # データ読み込み用
from tensorflow.keras.utils import to_categorical # データ読み込み用

# Inputレイヤーからスタート（返り値はテンソル）
inputs = Input(shape=(784,))

# レイヤークラスのインスタンスはテンソルを引数に取れる（返り値はテンソル）
x = Dense(128, activation='relu')(inputs)      # InputレイヤーとDenseレイヤー(1層目)を接続
x = Dense(64, activation='relu')(x)            # Denseレイヤー(1層目)とDenseレイヤー(2層目)を接続
output_layer = Dense(10, activation='softmax') # レイヤーのインスタンス化を切り分けることももちろん可能
                                               # (別のモデル構成時にこのレイヤーを指定・再利用することも可能になる)
predictions = output_layer(x)                  # Denseレイヤー(2層目)とDenseレイヤー(3層目)を接続

# Modelクラスを作成（入力テンソルと出力テンソルを指定すればよい）
model = Model(inputs=inputs, outputs=predictions) # これで、「(784,)のInputを持つDense3層」構成のモデルが指定される

# 以降はSequentialと同じ
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(x_train, y_train)

### Checkクイズ

* 問題1
    * ニューラル言語モデルの説明として最も正しいものを選びなさい
* 選択肢
  1. 自然言語による文章の識別確率をニューラルネットワークによってモデル化したもの  
  2. 自然言語による文章の識別確率をDenseレイヤーによってモデル化したもの  
  3. 自然言語による文章の生成確率をニューラルネットワークによってモデル化したもの  
  4. 自然言語による文章の生成確率をLSTMによってモデル化したもの


* 問題2
    * 分散表現の説明として誤っているものを選びなさい
* 選択肢
  1. one-hot表現は分散表現の1種である  
  2. 分散表現ではある1つの事柄がより普遍的な特徴の組み合わせとして表現される  
  3. 単語の分散表現は単語間の意味の近さが反映されたものになる  
  4. 分散表現を実現するアルゴリズムの1つとしてEmbeddingがある


* 問題3
    * 系列変換モデルの説明として誤っているものを選びなさい
* 選択肢
  1. 系列変換モデルはencorder-decorderモデルとも呼ばれる  
  2. 系列変換モデルはIMDBなどの文章感情分類といった分類タスクによく用いられる  
  3. 系列変換モデルは再帰レイヤーとしてRNNとLSTM以外も用いることができる  
  4. 系列変換モデルの符号化器の出力は利用してもしなくても構わない


* 問題4
    * KerasのFunctional APIやSequentialクラスの説明として最も正しいものを選びなさい
* 選択肢
  1. Functional APIによって構築されたモデルの学習はSequentialクラスによるモデルとは異なっている  
  2. Seq2SeqモデルをFunctional APIによって構築することはできない  
  3. 途中に分岐や合流があるような複雑なモデルはSequentialクラスによって構築することはできない  
  4. Functional APIによるモデル構築は自由度が高く、どんなレイヤーからはじめてもよい